In [1]:
# Notebook-level imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras



from sklearn.preprocessing import MinMaxScaler

# reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [2]:
# 1. Download historical OHLCV
TICKER    = "GOOG"
START     = "2012-01-01"
END       = "2022-12-21"
df        = yf.download(TICKER, START, END)
df.reset_index(inplace=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# 2. Compute moving averages
for w in (10, 50, 100, 200):
    df[f"MA_{w}"] = df["Close"].rolling(w).mean()


In [4]:
# 3. Compute RSI (14-day)
delta = df["Close"].diff()
gain  = delta.clip(lower=0)
loss  = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
df["RSI_14"] = 100 - (100 / (1 + rs))


In [6]:
# 4. Drop NaNs
df.dropna(inplace=True)
df.head()
# keep only features + target
features = ["Close", "MA_10","MA_50","MA_100","MA_200","RSI_14"]
data     = df[features].copy()

# 80/20 split
split_idx      = int(len(data) * 0.8)
train_df       = data.iloc[:split_idx]
test_df        = data.iloc[split_idx - 100:]   # overlap last 100 for sequences


In [7]:
# scale all features to [0,1]
scaler  = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df)
test_scaled  = scaler.transform(test_df)
def make_sequences(data_array, n_steps=100):
    X, y = [], []
    for i in range(n_steps, len(data_array)):
        X.append(data_array[i-n_steps:i])
        y.append(data_array[i, 0])         # predict scaled Close
    return np.array(X), np.array(y)

N_STEPS = 100
X_train, y_train = make_sequences(train_scaled, N_STEPS)
X_test,  y_test  = make_sequences(test_scaled,  N_STEPS)

print("Shapes:", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Shapes: (1949, 100, 6) (1949,) (513, 100, 6) (513,)


In [8]:
model = keras.models.Sequential([
    keras.layers.LSTM(64,  activation="tanh", return_sequences=True, input_shape=(N_STEPS, X_train.shape[2])),
    keras.layers.Dropout(0.2),
    
    keras.layers.LSTM(128, activation="tanh", return_sequences=False),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1,  activation="linear")
])

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=["mean_absolute_error"]
)

# callbacks: early stop + best-model checkpoint
es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
ck = keras.callbacks.ModelCheckpoint("best_stock_model.h5", save_best_only=True, monitor="val_loss")

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=32,
    callbacks=[es, ck],
    verbose=1
)

model.summary()


Epoch 1/100


c:\Users\Lenovo\Desktop\stocks\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0281 - mean_absolute_error: 0.1114

55/55 ━━━━━━━━━━━━━━━━━━━━ 12s 135ms/step - loss: 0.0278 - mean_absolute_error: 0.1107 - val_loss: 0.0132 - val_mean_absolute_error: 0.1010
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0056 - mean_absolute_error: 0.0524

55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 127ms/step - loss: 0.0056 - mean_absolute_error: 0.0523 - val_loss: 0.0053 - val_mean_absolute_error: 0.0583
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0047 - mean_absolute_error: 0.0478

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 64)        │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 363,461 (1.39 MB)

 Trainable params: 121,153 (473.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 242,308 (946.52 KB)

In [9]:
# ——— 7. Load Best Model & Predict on Test Set ———
# (ModelCheckpoint already saved the best weights to "best_stock_model.h5")
model.load_weights("best_stock_model.h5")

# 1. Predict (still scaled)
y_pred_scaled = model.predict(X_test)             # shape (n_samples, 1)

# 2. Invert scaling for the 'Close' feature (index 0)
#    data_min_ & data_range_ come from the MinMaxScaler on training data
data_min   = scaler.data_min_[0]
data_range = scaler.data_range_[0]

y_pred = y_pred_scaled.flatten() * data_range + data_min
y_actual = y_test * data_range + data_min

# 3. Compute evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_actual, y_pred)
mae = mean_absolute_error(y_actual, y_pred)
r2  = r2_score(y_actual, y_pred)

print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test R² : {r2:.4f}")

# ——— 8. Prepare Predictions DataFrame for Streamlit ———
# Map the predictions back to dates: 
# since test sequences started at df index split_idx, we align with df["Date"].iloc[split_idx:]
pred_dates = df["Date"].iloc[split_idx:].reset_index(drop=True)

results = pd.DataFrame({
    "Date":           pred_dates,
    "Actual_Close":   y_actual,
    "Predicted_Close": y_pred
})

# Peek at the first few rows
results.head()


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step
Test MSE: 81.9145
Test MAE: 7.4690
Test R² : 0.7546


,Date,Actual_Close,Predicted_Close
0,2020-12-08,90.498856,88.932300
1,2020-12-09,88.785965,89.089884
2,2020-12-10,88.348038,89.079639
3,2020-12-11,88.668526,88.868651
4,2020-12-14,87.588142,88.879111
